# Cifar10 model testing
In this notebook we will test if working with grey-scaled images gives better results than working with RGB images. For this, we will try several techniques:
- Flatten the three channels **(Flatten)**
- Maximum of the three channels **(MaxChannels)**
- Use the image with three channels **(Embedding)**
- Convolutional layer **(Convolutional)**
- Black and white function **(BlackWhite)**

# Make the necessary imports

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
!pip install umap-learn[plot]

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from classification_models.tfkeras import Classifiers
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import math
import os
import gc
import cv2

import umap.umap_ as umap
import umap.plot

# Import and preprocess the Cifar10 dataset

In [ ]:
(Xtrain_orig, ytrain_orig), (Xtest_orig, ytest_orig) = cifar10.load_data()
Xtrain_orig = Xtrain_orig.astype('float32')
Xtest_orig = Xtest_orig.astype('float32')

print ("Xtrain shape: " + str(Xtrain_orig.shape))
print ("ytrain shape: " + str(ytrain_orig.shape))

print ("Xtest shape: " + str(Xtest_orig.shape))
print ("ytest shape: " + str(ytest_orig.shape))

# Support functions

In [ ]:
def create_random_set(X, y, strip_size=4, set_size=25000):
    
    #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We want the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 3])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, 0] = tf.squeeze(X[X_groups[i][j]])
            X_processed[i, j:j+1, :, :, 1] = tf.squeeze(X[X_groups[i][j]])
            X_processed[i, j:j+1, :, :, 2] = tf.squeeze(X[X_groups[i][j]])
        X_processed[i, strip_size, :, :, 0] = tf.squeeze(X[X_groups[i][strip_size]])
        X_processed[i, strip_size, :, :, 1] = tf.squeeze(X[X_groups[i][strip_size]])
        X_processed[i, strip_size, :, :, 2] = tf.squeeze(X[X_groups[i][strip_size]])
        y_processed[i] = y_label[i]
        
    return X_processed, y_processed

In [ ]:
def create_random_set_RGB(X, y, strip_size=4, set_size=60000):
    
        #Create a list of lists where every sublist contains the indexes of the images belonging to a class
    list_indices_by_number = [np.where(y == i)[0] for i in range(10)]
    
    #Create the strips of images
    X_groups = []
    number_groups = []
    y_label = []
    
    for i in range(set_size): #Create as many images as strip_size
        group_i = []
        numbers_i = []
        while len(group_i) < strip_size: #While the strip is shorter that the size wanted
            #Choose a random index
            image_idx = random.randint(0, len(X)-1)
            numbers_i.append(y[image_idx][0])
            group_i.append(image_idx)
        #When the strip is full, add the target image. Use random to obtain a balanced set.
        repeated = np.random.choice([0, 1], p=[0.50, 0.50])
        if repeated:
            #Look for a number whose class is already contained in the strip.
            random_idx = random.randint(0, len(numbers_i)-1)
            number = numbers_i[random_idx]
            numbers_i.append(number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[number])-1)
            group_i.append(list_indices_by_number[number][image_idx])
            y_label.append(1)
        else:
            #Add a number that is not aready in the strip
            possible_numbers = [x for x in range(10) if x not in numbers_i]
            random_number = random.choice(possible_numbers)
            numbers_i.append(random_number)
            #Choose a random image representing the chosen class
            image_idx = random.randint(0, len(list_indices_by_number[random_number])-1)
            group_i.append(list_indices_by_number[random_number][image_idx])
            y_label.append(0)
        X_groups.append(group_i)
        number_groups.append(numbers_i)
    
    #We want the following shape: (N, strip_size+1, X_train[1], X_train[2], 3)
    #And create the labels
    N = len(X_groups)
    img_size1 = X.shape[1]
    img_size2 = X.shape[2]
    X_processed= np.zeros([N, strip_size+1, img_size1, img_size2, 3])
    y_processed = np.zeros([N])
    for i in range(N):
        numbers_i = list(dict.fromkeys(number_groups[i]))
        for j in range(strip_size):
            X_processed[i, j:j+1, :, :, :] = X[X_groups[i][j]]
        X_processed[i, strip_size, :, :, :] = X[X_groups[i][strip_size]]
        y_processed[i] = y_label[i]
        
    return X_processed, y_processed

In [ ]:
def create_random_data_sets(X, y, Xt, yt, strip_size, RGB, training_size=30000, test_size=2000):
    if(RGB):
        Xtrain, ytrain = create_random_set_RGB(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.1)
        Xtest, ytest = create_random_set_RGB(Xt, yt, strip_size, set_size=test_size)
    else:
        Xtrain, ytrain = create_random_set(X, y, strip_size, set_size=training_size)
        Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.1)
        Xtest, ytest = create_random_set(Xt, yt, strip_size, set_size=test_size)
    
    print ("Training examples classified as 0: " + str(len(np.where(ytrain==0)[0])))
    print ("Training examples classified as 1: " + str(len(np.where(ytrain==1)[0])))
    print ("Validation examples classified as 0: " + str(len(np.where(yval==0)[0])))
    print ("Validation examples classified as 1: " + str(len(np.where(yval==1)[0])))
    print ("Test examples classified as 0: " + str(len(np.where(ytest==0)[0])))
    print ("Test examples classified as 1: " + str(len(np.where(ytest==1)[0])))
    
    return Xtrain, Xval, Xtest, ytrain, yval, ytest

In [ ]:
def train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, lr=1e-3, batch_size=32, model_save_name="best_model"):

    #Define callbacks
    #Save the best model
    dirname = os.getcwd()
    filepath = os.path.join(dirname, model_save_name)
    filepath = os.path.join(filepath, 'model')
    
    model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss',
        mode='min', verbose = 0, save_best_only=True, save_weights_only=True)
    #Add early stopping
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, verbose = 0)
    #Reduce learning rate on plateau
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=5e-5)
    callbacks = [model_checkpoint_cb, early_stopping_cb, reduce_lr]

    #Compile and fit the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    history = model.fit(Xtrain, ytrain,
                        batch_size=batch_size,
                        epochs=100,
                        validation_data=(Xval, yval),
                        callbacks=callbacks,
                        verbose=1)
    
    """plt.figure(figsize=(12,6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Val'], loc='upper right')


    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Val'], loc='upper right')"""
    
    model.load_weights(filepath)
    ypredict = model.predict(Xtest)
    #ypredict = tf.squeeze(ypredict).numpy()
    #print(ypredict)
    #ypredict_round = [round(x) for x in ypredict]
    score = model.evaluate(Xtest, ytest, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    
    #cm = confusion_matrix(ytest, ypredict)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    #disp.plot()
    #plt.show()
    
    return score[1], model

In [ ]:
def gray_to_rgb(img):
    return np.repeat(img, 3, 2)

In [ ]:
def see_embeddings(model, X, y, save_name, preprocess_func, pre_model):
    embeddings_per_class = 10
    classes = 10
    # Buscamos 3 imagenes de cada clase
    #Creamos una lista de listas donde cada sublista contiene los índices de la imágenes de un número
    list_indices_by_number = [np.where(y == i)[0] for i in range(classes)]
    
    images = np.zeros((classes*embeddings_per_class, X.shape[1], X.shape[2], X.shape[3]))
    for i in range(classes):
        list_of_indexes = random.choices(list_indices_by_number[i], k=embeddings_per_class)
        images[embeddings_per_class*i:(i+1)*embeddings_per_class, :, :, :] = X[list_of_indexes]

    if preprocess_func:
        prep_images = preprocess_func(images)
        prep_X = preprocess_func(X)
    else:
        prep_images = images
        prep_X = X

    if pre_model:
        prep_images = model.preprocess(prep_images)
        prep_X = model.preprocess(prep_X)
    
    if prep_X.shape[-1] != 3:
        prep_images = tf.repeat(prep_images, 3, axis=-1)
        prep_X = tf.repeat(prep_X, 3, axis=-1)

    encoded_images = model.encoder(prep_images).numpy()
    encoded_X = model.encoder(prep_X).numpy()
    
    plt.figure(figsize=(15, 30))

    m = embeddings_per_class
    n = classes
    for i in range(m):
        for j in range(n):
            ax = plt.subplot(m, n, n*i+j+1)
            plt.imshow(tf.expand_dims(encoded_images[n*i+j], axis=-1))
            plt.title('Class: ' + str(i), fontsize=16)
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
    plt.suptitle('Image embeddings per class', fontsize=20)
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    plt.savefig(save_name+ '.png')
    plt.show()
    
    manifold = umap.UMAP().fit(encoded_X)
    p = umap.plot.points(manifold, labels=tf.squeeze(y), color_key_cmap='Paired', background='black')
    plt.savefig(save_name+'_UMAP'+ '.png')
    umap.plot.plt.show()
    

In [ ]:
def save_to_db(db_name, column_name, list_to_save):
    df = pd.read_csv(db_name + '.csv')
    df[column_name] = list_to_save
    df.to_csv(db_name + '.csv', index=False)

def create_db(db_name):
    df = pd.DataFrame()
    df.to_csv(db_name + '.csv', index=True)

def plot_db_columns(db_name, title, xlabel, ylabel, save_name):
    df = pd.read_csv(db_name + '.csv')
    # plot lines
    x = [i for i in range(2, 11)]
    plt.figure(figsize=(9,7))
    
    for column in df:
        if (column != 'Unnamed: 0'):
            plt.plot(x, df[column], label = column)

    plt.title(title, fontsize=20)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.legend()
    plt.grid(axis = 'y', color = 'gray', linestyle = '--', linewidth = 0.5)
    plt.tick_params(labelsize=14)
    plt.savefig(save_name + '.png')
    plt.show()

In [ ]:
def run_model(X, y, Xt, yt, model_class, latent_dim, model_save_name, db_name, column_name, preprocess_func, preprocess_before, num_iterations=3, RGB=True, pre_model=False):
    
    #Preprocess data
    if preprocess_before:
        X_processed = preprocess_func(X)
        Xt_processed = preprocess_func(Xt)
    else:
        X_processed = X
        Xt_processed = Xt
        
    accuracy_per_strip_size = []
    for strip_size in range(1, 10):
        print('-------------------' + str(strip_size) + '-------------------')
        channels = strip_size + 1
        accuracy_per_iteration = []
        for i in range(num_iterations):
            print('--------------Iteration ' + str(i+1) + '--------------')
            Xtrain, Xval, Xtest, ytrain, yval, ytest = create_random_data_sets(X_processed, y, Xt_processed, yt, strip_size, RGB)
            tf.keras.backend.clear_session()
            model = model_class(latent_dim, channels, (None, Xtrain[0].shape[1], Xtrain[0].shape[2], Xtrain[0].shape[3])) 
            score, model = train_model(model, Xtrain, ytrain, Xval, yval, Xtest, ytest, batch_size=64, model_save_name=model_save_name)
            if i == (num_iterations-1):
                embedding_file_path = os.path.join('embedding_images', model_save_name + "_" + str(channels))
                see_embeddings(model, Xt, yt, embedding_file_path, preprocess_func, pre_model)
            accuracy_per_iteration.append(score)
            del Xtrain
            del Xval
            del Xtest
            del ytrain
            del yval
            del ytest
            gc.collect()
        mean_score = np.mean(accuracy_per_iteration)
        accuracy_per_strip_size.append(mean_score)
    save_to_db(db_name, column_name, accuracy_per_strip_size)

# Check if the strips are correctly built

In [ ]:
strip_size = 5

Xtrain, ytrain = create_random_set_RGB(Xtrain_orig/255., ytrain_orig, strip_size, 500)

print ("Training examples classified as 0: " + str(len(np.where(ytrain==0)[0])))
print ("Training examples classified as 1: " + str(len(np.where(ytrain==1)[0])))

#First show one with expected output 0
idx = np.where(ytrain==0)[0][3]
f, axarr = plt.subplots(1,strip_size+1)
for i in range(strip_size+1):
    axarr[i].imshow(Xtrain[idx, i, :, :, :], cmap='gray')

#Now one with expected output 1
idx = np.where(ytrain==1)[0][0]
f, axarr = plt.subplots(1,strip_size+1)
for i in range(strip_size+1):
    axarr[i].imshow(Xtrain[idx, i, :, :, :], cmap='gray')

#Now with black and white images
Xtrain_bw = tf.image.rgb_to_grayscale(Xtrain_orig)
Xtrain, ytrain = create_random_set(Xtrain_bw/255., ytrain_orig, strip_size, 500)

print ("Training examples classified as 0: " + str(len(np.where(ytrain==0)[0])))
print ("Training examples classified as 1: " + str(len(np.where(ytrain==1)[0])))

#First show one with expected output 0
idx = np.where(ytrain==0)[0][0]
f, axarr = plt.subplots(1,strip_size+1)
for i in range(strip_size+1):
    axarr[i].imshow(Xtrain[idx, i, :, :, :], cmap='gray')

#Now one with expected output 1
idx = np.where(ytrain==1)[0][0]
f, axarr = plt.subplots(1,strip_size+1)
for i in range(strip_size+1):
    axarr[i].imshow(Xtrain[idx, i, :, :, :], cmap='gray')

del Xtrain
del Xtrain_bw
del ytrain

# Define the models

In [ ]:
def preprocess_identity(images):
    return images

## Modelos con EfficientNetB0 encoder

In [ ]:
def build_efficientnet_encoder(shape_in, latent_dim):
  
    inp = layers.Input(shape=shape_in)
 
    # Apply EfficientNet to every image
    efficientnet = keras.applications.EfficientNetB0(include_top=False, pooling='avg', weights='imagenet', drop_connect_rate=.5)
    x = efficientnet(inp)
    
    x = layers.Dropout(.8)(x)
    x = layers.Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    x = layers.Dropout(.5)(x)
    # Embedding de cada imatge (dim reduced from 2048 to encoding_dim)
    x = layers.Dense(latent_dim, activation='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
    encoder = keras.Model(inputs=inp, outputs=x)

    return encoder

#from keras.utils.vis_utils import plot_model
#plot_model(buildEfficientNet((32, 32, 3), 32), to_file='efficientnet_encoder.png', show_shapes=True, show_layer_names=False)
#plot_model(tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3), pooling='avg', drop_connect_rate=0.4), to_file='efficientnet.png', show_shapes=True, show_layer_names=True)

In [ ]:
class DotEfficientnet(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(DotEfficientnet, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = build_efficientnet_encoder(self.shape_in[1:], self.latent_dim)

    def call(self, x):
      
        encoded_images = layers.TimeDistributed(self.encoder)(x)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        last_embedding = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])
        return y_predict

In [ ]:
class ClassifierEfficientnet(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(ClassifierEfficientnet, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = build_efficientnet_encoder(self.shape_in[1:], self.latent_dim)
        
        self.classifier = tf.keras.Sequential([
            layers.Flatten(),
            #layers.Dense(512, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def call(self, x):
      
        encoded_images = layers.TimeDistributed(self.encoder)(x)
        
        if(self.channels > 2):
            max_image = layers.Maximum()([layers.Lambda(lambda x : x[:,i,:])(encoded_images) for i in range(self.channels-1)])
        else:
            max_image = layers.Lambda(lambda x : x[:,0,:])(encoded_images)
            
        target_image = layers.Lambda(lambda x : x[:,-1,:])(encoded_images)
        stacked_image = tf.stack([max_image, target_image], axis=-1)
        y_predict = self.classifier(stacked_image)
        
        return y_predict

## Models with Resnet18

In [ ]:
def buildResNet(img_size, latent_dim):
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    base_model = ResNet18(img_size, weights='imagenet', include_top=False, pooling='avg', drop_connect_rate=0.4)

    x = base_model.output
    x = layers.Flatten()(x)
    x = layers.Dropout(.3)(x)
    predictions = layers.Dense(latent_dim, activation='relu')(x)
    
    return keras.models.Model(inputs=base_model.input, outputs=predictions)

#plot_model(buildResNet((32, 32, 3), 32), to_file='resnet_encoder.png', show_shapes=True, show_layer_names=False)
#ResNet18, preprocess_input = Classifiers.get('resnet18')
#plot_model(ResNet18((32, 32, 3), weights='imagenet', include_top=False, pooling='avg', drop_connect_rate=0.4), to_file='resnet.png', show_shapes=True, show_layer_names=False)

In [ ]:
class Resnet_Model(keras.Model):
    def __init__(self, latent_dim, channels, shape_in):
        super(Resnet_Model, self).__init__()
        self.latent_dim = latent_dim
        self.channels = channels
        self.shape_in = shape_in
        
        self.encoder = buildResNet(self.shape_in, self.latent_dim)

    def call(self, x):
      
        encoded_images_list = []
        for i in range(self.channels-1):
            channel_i = layers.Lambda(lambda x : x[:,:,:,i:i+(3)])(x)
            encoded_images_list.append(self.encoder(channel_i))
        
        if(self.channels > 2):
            max_image = layers.Maximum()(encoded_images_list)
        else:
            max_image = encoded_images_list[0]
        
        
        last_channel = layers.Lambda(lambda x : x[:,:,:,self.channels-1: self.channels+(3)-1])(x)
        last_embedding = self.encoder(last_channel)
        
        y_predict = layers.Dot(axes=1, normalize=True)([max_image, last_embedding])

        return y_predict

# Train models

In [ ]:
os.mkdir('embedding_images')

In [ ]:
create_db('cifar10_efficientnet_encoder')

In [ ]:
#Modelos con EfficientNetB0 de encoder y dot product
run_model(Xtrain_orig, ytrain_orig, Xtest_orig, ytest_orig, DotEfficientnet, 32, 'embedding_dot_efficient32', 'cifar10_efficientnet_encoder', 'embedding_dot_efficient32', preprocess_identity, True, num_iterations=1, RGB=True)


In [ ]:
run_model(Xtrain_orig, ytrain_orig, Xtest_orig, ytest_orig, DotEfficientnet, 10, 'embedding_dot_efficient10', 'cifar10_efficientnet_encoder', 'embedding_dot_efficient10', preprocess_identity, True, num_iterations=1, RGB=True)

In [ ]:
#Modelos con EfficientNetB0 de encoder y dot product
run_model(Xtrain_orig, ytrain_orig, Xtest_orig, ytest_orig, ClassifierEfficientnet, 32, 'embedding_classifier_efficient32', 'cifar10_efficientnet_encoder', 'embedding_classifier_efficient32', preprocess_identity, True, num_iterations=1, RGB=True)


In [ ]:
run_model(Xtrain_orig, ytrain_orig, Xtest_orig, ytest_orig, ClassifierEfficientnet, 10, 'embedding_classifier_efficient10', 'cifar10_efficientnet_encoder', 'embedding_classifier_efficient10', preprocess_identity, True, num_iterations=1, RGB=True)


# Plot results

In [ ]:
plot_db_columns('cifar10_efficientnet_encoder', 'Cifar10 model accuracy', 'Strip size', 'Accuracy', 'cifar10_efficientnet_encoder')
